In [ ]:
import sys
import os

sys.path.append('../')
from dataloader.MiddleburyLoader import myImageFloder
from notebooks.notebook_utils import visualize_sample, download_file_s3, list_s3

In [ ]:
train_list = list_s3('hdsm/kitti12/training/disp_occ/', delim='')['Contents']

In [ ]:
def get_all_keys(training_folder, base_name):
    li = training_folder + '/colored_0/{}.png'.format(base_name)
    ri = training_folder + '/colored_1/{}.png'.format(base_name)
    ld = training_folder + '/disp_occ/{}.png'.format(base_name)
    return li, ri, ld

def get_out_file(s3_key):
    s3_key_split = s3_key.split('/')
    out_folder = '../data/{}'.format('/'.join(s3_key_split[1:-1]))
    os.makedirs(out_folder, exist_ok=True)
    out_file = os.path.join(out_folder, '{}'.format(s3_key_split[-1]))
    return out_file

def walk_through_dataset(dataset_list):
    for example in dataset_list:
        disp_key = example['Key']
        if '_10.png' not in disp_key:
            continue
        base_name = os.path.basename(disp_key)[:-4]
        training_folder = '/'.join(disp_key.split('/')[0:-2])
        li, ri, ld = get_all_keys(training_folder, base_name)
        
        all_left = [get_out_file(li)]
        all_right = [get_out_file(ri)]
        all_left_disp = [get_out_file(ld)]
        
        download_file_s3(li, all_left[0])
        download_file_s3(ri, all_right[0])
        download_file_s3(ld, all_left_disp[0])
        
        rand_scale = [0.9, 2.4]
        loader_kitti12 = myImageFloder(all_left, all_right, all_left_disp, rand_scale=rand_scale, order=0)
        for (img_l, img_r, disp_l) in loader_kitti12:
            visualize_sample(img_l, img_r, disp_l)
            yield

In [ ]:
train_iter = walk_through_dataset(train_list)

In [ ]:
next(train_iter)